# Build de un proyecto

Para hacer el build del proyecto, de una manera estandar moderna utilizamos 2 elementos:

- pyproject.toml
- setup.py

Puede utilizarse setup.py solamente, ya que ese era el estandar hace unos anios. De todas formas lo que hacemos ahora es usar pyproject.toml y setup.py la reservamos para cuando necesitemos agregar extensiones para nuestros codigos de C por ejemplo.

## Construccion de una extension C en el proyeto

Describimos la situacion del caso:

- El codigo C se compila en .so y se utiliza por medio de una interfaz de ctypes.
- Sin embargo lo que se necesita es que al momento de instalar el proyecto se compilen estos objetos en el lugar deseado.
- Se cuenta con un MAKEFILE para compilar los objetos.

### Estructura de setup.py y setuptools

En principio esta estructura es compleja. Asi que vamos a hablar de dos elementos de setuptools. Setuptools es el modulo moderno para hacer cosas en la instalacion usando setup.py. Antes se tenia distutils por las dudas, ahora setuptools constituye una mejora de los elementos de distutils.

El archivo setup.py debe ir afuera de la carpeta del proyecto, junto con otros archivos como pyproject.toml.

Los elementos de setuptools de los que vamos a hablar son:

- setup : *setuptools.setup() is the main function in setup.py used to define and configure a Python package for distribution. It provides metadata about the package and specifies how it should be built, installed, and distributed.* . Esta funcion siempre estara presente en setup.py y concretamente la utilizaremos para generar nuestros ejecutables. Sin embargo, aunque puede indicarse elementos sobre el paquete, como autor, version, etc, esas las dejamos para pyproject.toml. Veamos algunos usos que daran mas contexto a lo siguiente:

  - Defines package name, version, author, and description. # No lo usaremos ya en pyproject.toml
  - Lists dependencies (install_requires). # No lo usaremos ya en pyproject.toml
  - Configures C extensions (ext_modules)
  - Allows custom build commands (cmdclass).
  - Enables packaging for PyPI (sdist, bdist_wheel)

- Extension : *Extension from setuptools is a class used to define C or C++ extensions in a Python package. It specifies how to compile and link source code files (typically .c or .cpp) into shared object (.so) or dynamic link library (.dll) files that can be used by Python modules.*. Elementos clave:

  - name: The name of the extension, which should match the module you want to create (e.g., my_module).,
  - sources: A list of C/C++ source files to compile into the extension (e.g., ["src/my_module.c"]).
  - extra_compile_args: Additional flags or arguments passed to the compiler (e.g., ["-fPIC", "-shared"]).

```python

from setuptools import setup, Extension

# Define a C extension
extension = Extension(
name="voidfindertk.zobov.zobov_loader",  # Name of the resulting module
sources=["voidfindertk/zobov/zobov_loader.c"],
extra_compile_args=["-fPIC"]
)

setup(
    ext_modules=[extension],  # Include extension in the package
)

from setuptools import setup, Extension<br><br>

# Define a C extension
extension = Extension(
name="voidfindertk.zobov.zobov_loader",  # Name of the resulting module
sources=["voidfindertk/zobov/zobov_loader.c"],
extra_compile_args=["-fPIC"]
)

setup(
    ext_modules=[extension],  # Include extension in the package
) 
```
- build_ext : *setuptools.build_ext is a command class in setuptools used to build C or C++ extensions in a Python package. It is part of the build process and is typically invoked when you run python setup.py build_ext or python setup.py install. This command compiles C or C++ source files into shared libraries (e.g., .so, .dll) that can be imported and used by Python.* Esto permite compilar nuestros modulos c con cython (aunque los .so salen con un nombre de ejecutable vinculado a la pc del usuario). Concretamente, build_ext es una clase, de la cual podemos heredar (Ver el ejemplo mas abajo)

  - Si nuestra clase hereda de build_ext, luego en setup() debemos invocar el argumento: cmdclass={"build_ext": < clase custom que hereda de build_ext>},
  - Esto se utiliza para poder compilar ejecutables



- build : *setuptools.build is a command class in setuptools that is responsible for the general build process of a Python package. It is invoked when you run python setup.py build, and it handles the overall construction of the package before it is installed, distributed, or packed into an archive (such as a .tar.gz or .whl file).*

  - En general esto no es lo que hariamos para compilar nuestro codigo c, ya que se delega para otras tareas, sin embargo, para correr un makefile, esta seria nuestra opcion.
  - Como en build_ext, debemos generar una clase que herede de build y cuyos metodos se ejecutaran al llamar a setup(cmdclass)
  - En setup debemos agregar: cmdclass={"build": < clase que hereda de build>}

### NOTA IMPORTANTE

- Tener en cuenta de verificar que en MANIFEST.in, se incluya los archivos .so, de otra manera , si por ejemplo esta la sentencia: recursive-exclude *.so , lo que se compile no se incluira en el paquete.

## Probando las configuraciones de setup.py

<table border="1">
  <thead>
    <tr>
      <th>Command</th>
      <th>Purpose</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>python setup.py check</td>
      <td>Check syntax & metadata.</td>
    </tr>
    <tr>
      <td>python setup.py build_ext --inplace</td>
      <td>Test compiling .so files.</td>
    </tr>
    <tr>
      <td>python setup.py build</td>
      <td>Test full package build.</td>
    </tr>
    <tr>
      <td>python setup.py install</td>
      <td>Install package locally.</td>
    </tr>
    <tr>
      <td>pip install -e .</td>
      <td>Install in development mode.</td>
    </tr>
    <tr>
      <td>python setup.py sdist bdist_wheel</td>
      <td>Prepare for PyPI distribution.</td>
    </tr>
  </tbody>
</table>

In [ ]:
from setuptools import setup, Extension
from setuptools.command.build_ext import build_ext
from setuptools.command.build import build
import pathlib, shutil, os, subprocess

# Ejemplo de aplicacion de build_ext

class CustomBuildExt(build_ext):
    def build_extension(self, ext):
        """Modify the build process to control the output filename."""
        output_path = self.get_ext_fullpath(ext.name)  # Default output path
        output_path = output_path.rsplit('.', 1)[0] + ".so"  # Remove Python-specific suffix

        obj_files = [self.compiler.compile(ext.sources)[0]]  # Compile sources
        self.compiler.link_shared_object(obj_files, output_path)  # Link shared object

extensions = [
    Extension(
        name="voidfindertk.zobov.zones_in_void",
        sources=["voidfindertk/zobov/zones_in_void.c"],
        extra_compile_args=["-fPIC"],
    ),
]

setup(
    ext_modules=extensions,
    cmdclass={"build_ext": CustomBuildExt},
)

# Ejemplo de aplicacion de build

class MakeRun(build):
    def run(self):
        # Run `make` before the build process
        makefile_dir = os.path.join(os.path.dirname(__file__), "voidfindertk/zobov")
        try:
            subprocess.check_call(["make"], cwd=makefile_dir)
        except subprocess.CalledProcessError as e:
            raise RuntimeError(f"Make failed with error: {e}")
        
        # Continue with the normal build process
        super().run()


setup(
    cmdclass={"build": MakeRun}
)


In [ ]:
import matplotlib_inline.pyplot as plt
import numpy as np

x = np.arange(1000,10000,10)
y = f(x)
z = -x/2

fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.plot(x,y,'o--',ms=0.5)
ax.plot(x,z,'o--',ms=0.5, c='red')
plt.savefig("plot.jpg")
plt.close()
